<a href="https://colab.research.google.com/github/Yanlin126/images_eye-size_project/blob/main/eye_size_measurements.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import pandas as pd

In [ ]:
# Step 1: Load the sample image
from google.colab import drive
import os

# Mount Google Drive to access files
drive.mount('/content/gdrive')

# Specify the name of the uploaded folder containing the images
uploaded_folder = 'image_eye_size_project'

# Combine the Google Drive path with the uploaded folder path
image_folder = os.path.join('/content/gdrive/My Drive', uploaded_folder)

# Get a list of image files in the folder
image_files = os.listdir(image_folder)

# Print the list of image files
print(image_files)



Mounted at /content/gdrive
['2016 01 13   LR 24H S1 control 3X.JPG', '2022 12 14   RA LR 24H S3 3X.JPG', '2016 01 12   LR 24H S1 control 3X.JPG', '2016 01 13   LR 24H S1 3X.JPG', ' 2022 11 26 100% DMSO LR 24H S1 control 3X.JPG', '2022 12 14   RA LR 24H S1 3X.JPG', '2022 03 20   LR 24H S2 3X.JPG', '2015 02 24   LR 24H S2 control 3X.JPG', '2022 04 27   LR 24H S1 3X.JPG', '2022 12 13 100% DMSO LR 24H S4 3X.JPG', '2015 02 24   LR 24H S4 control 3X.JPG', ' 2022 10 06 100% DMSO LR 24H S2 control 3X.JPG', '2022 12 13 100% DMSO LR 24H S2 control 3X.JPG', '2015 02 24   LR 24H S5 control 3X.JPG', '2022 12 13 100% DMSO   LR 24H S1 3X.JPG', ' 2022 10 06 100% DMSO LR 24H  S1 control 3X.JPG', '2022 12 13 100% DMSO LR 24H S5 control  3X.JPG', '2022 03 20   LR 24H S2 control 3X.JPG', '2015 02 24   LR 24H S4 3X.JPG', '2022 12 14   RA LR 24H S2 control 3X.JPG', '2015 02 24   LR 24H S2 3X.JPG', '2016 01 13   LR 24H S2 control 3X.JPG', '2022 12 14   RA LR 24H S2 3X.JPG', '2022 12 13 100% DMSO LR 24H S4 co

In [ ]:
# Step 2: Define preprocessing functions
def resize_image(image, target_size):
    return cv2.resize(image, target_size)

def denoise_image(image):
    return cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 21)


In [27]:
# Step 3: Batch processing
for filename in image_files:
    image_path = os.path.join(image_folder, filename)
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)

    # Step 4: Apply preprocessing functions
    resized_image = resize_image(image, (500, 500))
    denoised_image = denoise_image(resized_image)


    # Step 5: Save the preprocessed image (optional)
    # You can save the preprocessed images to a new folder if needed
    destination_folder = '/content/gdrive/My Drive/denoised image of the eye size project/'
    output_filename = filename.replace('.JPG', '_denoised.jpg')  # Replace '.JPG' with the appropriate file extension of your original images
    output_path = os.path.join(destination_folder, output_filename)
    cv2.imwrite(output_path, denoised_image)



error: ignored

In [ ]:
# Step 3: Image Thresholding
# Replace 'threshold_value' with an appropriate value that suits your images
_, binary_image = cv2.threshold(image, threshold_value, 255, cv2.THRESH_BINARY)


In [ ]:
# Step 4: Contour Detection
contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [ ]:
# Step 5: Diameter Measurement
measurements = []
for contour in contours:
    # Fit an ellipse to the contour
    ellipse = cv2.fitEllipse(contour)
    center, (major_axis, minor_axis), angle = ellipse

    # Calculate horizontal and vertical diameters
    horizontal_diameter = major_axis
    vertical_diameter = minor_axis

    # Calculate average diameter
    average_diameter = (horizontal_diameter + vertical_diameter) / 2

    # Calculate area
    area = np.pi * (horizontal_diameter / 2) * (vertical_diameter / 2)

    # Store measurements in a list
    measurements.append([horizontal_diameter, vertical_diameter, average_diameter, area])

In [ ]:
# Step 6: Create a DataFrame and save measurements to CSV
df = pd.DataFrame(measurements, columns=['Horizontal Diameter', 'Vertical Diameter', 'Average Diameter', 'Area'])
df.to_csv('eye_measurements.csv', index=False)